In [ ]:
# assume working in Jupyter Lab
#%matplotlib widget

%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy.optimize import curve_fit 
from tqdm.notebook import tqdm

In [ ]:
sns.set_style('whitegrid')
sns.set_context('paper', font_scale=1.4)
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

# Importing the database

In [ ]:
# Importing Resumed Parameters
data = pd.read_csv('WEST_C5_database_resumed_parameters.csv')
data

# Cleaning the data

Below we remove non physical or not relevant values

In [ ]:
len(data)

In [ ]:
data = data.query('nl > 2 and Ip > 0.4')\
            .query('Rext_median > 2700')

Total number of points:

In [ ]:
len(data)

Names of available data in the dataset:

In [ ]:
data.columns

In [ ]:
first_C5_pulse = 56247
january_restart = 56544
post_boro = [56247, 56597]
phase_control_OK = 56469
Q2_position_OK = 56737

In [ ]:
sessions = {
    'C1-C5-W1-21': {
        'name': 'Impact of BN tiles on W Sources during start up/divertor phase - Ohmic',
        'pulses': (56415,56435),
        'date': ('2020-12-16')
    },
    'C1-C5-W1-23': {
        'name': 'Impact of BN tiles on W Sources during divertor phase - Heating',
        'pulses': (56462,56489),
        'date': ('2020-12-18')
    },
    'C1-C5-W1-11': {
        'name': 'Melting experiments',
        'pulses': (56516,56543),
        'date': ('2020-12-22')
    },
    'C1-C5-W1-10': {
        'name': 'Weight of W divertor sources vs main chamber W sources',
        'pulses': [(56803,56819), (56850,56860)],
        'date': ('2021-01-21', '2020-01-22')
    },
    'C1-C5-W1-22': {
        'name': 'Divertor exhaust enhancement studies with the IPD',
        'pulses': [(56820,56833), (56913,56927)],
        'date': ('2020-01-21', '2020-01-27')
    },
    'C1-C5-W1-25': {
        'name': 'Impurity Powder dropper commisionning',
        'pulses': (56635,56640),
        'date': ('2021-01-12')
    },
    'C1-C5-W1-24': {
        'name': 'Heat load on misalignmed PFU/gaps',
        'pulses': (56670,56698),
        'date': ('2021-01-14')
    },
}

In [ ]:
def position_antennes(pulse):
    print('Q1:', data.query(f'pulse == {pulse}')['R_Q1'].unique())
    print('Q2:', data.query(f'pulse == {pulse}')['R_Q2'].unique())
    print('Q4:', data.query(f'pulse == {pulse}')['R_Q4'].unique())
    print('LH1:', data.query(f'pulse == {pulse}')['R_LH1'].unique())
    print('LH2:', data.query(f'pulse == {pulse}')['R_LH2'].unique())
    
position_antennes(56737)

In [ ]:
data.query('nl < 4 and IC_P_tot > 0.2')['pulse'].unique()

# Derived quantities

Below we calculate some quantities derived from the others

In [ ]:
# Radiated/Convected powers & radiated power fraction
data['P_RF'] = data['LH_P_tot'] + data['IC_P_tot']
data['Ptot'] = data['Ohmic_P'] + data['P_RF']
data['P_conv'] = data['Ptot'] - data['Prad']
data['frad'] = data['Prad']/data['Ptot']

# Normalized data
data['Cu_norm'] = data['Cu']/data['nl']
data['Fe_norm'] = data['Fe']/data['nl']
data['Ag18_norm'] = data['Ag18']/data['nl']
data['Ag19_norm'] = data['Ag19']/data['nl']
data['Ag18_norm_nl_IC'] = data['Ag18']/data['nl']/data['IC_P_tot']
data['Ag19_norm_nl_IC'] = data['Ag19']/data['nl']/data['IC_P_tot']

data['Cu_norm_nl_IC'] = data['Cu_norm']/data['IC_P_tot']
data['Cu_norm_nl_LH'] = data['Cu_norm']/data['LH_P_tot']
data['Cu_norm_nl_RF'] = data['Cu_norm']/data['P_RF']

data['ratio_Pic_Plh'] = data['IC_P_tot']/data['LH_P_tot']
data['Isotopic Ratio INBUM04_norm_PLH'] = data['Isotopic Ratio INBUM04']/data['LH_P_tot']
data['Isotopic Ratio LODIVOU15_norm_PLH'] = data['Isotopic Ratio LODIVOU15']/data['LH_P_tot']

data['Isotopic Ratio INBUM04_norm_PRF'] = data['Isotopic Ratio INBUM04']/data['P_RF']
data['Isotopic Ratio LODIVOU15_norm_PRF'] = data['Isotopic Ratio INBUM04']/data['P_RF']

data['IC_Q1_Rc_left_over_right'] = data['IC_Rc_Q1_left']/data['IC_Rc_Q1_right']
data['IC_Q2_Rc_left_over_right'] = data['IC_Rc_Q2_left']/data['IC_Rc_Q2_right']
data['IC_Q4_Rc_left_over_right'] = data['IC_Rc_Q4_left']/data['IC_Rc_Q4_right']

data['pulse_relative'] = data['pulse'] - first_C5_pulse

# LSN and USN shots
data['Xpoint'] = 'LSN'
data.loc[(data['pulse'] >= 56737) & (data['pulse'] <=56769), 'Xpoint'] = 'USN' # 2021-01-19
data.loc[(data['pulse'] >= 56770) & (data['pulse'] <=56802), 'Xpoint'] = 'USN' # 2021-01-20

# objet le plus avancé
data['R_obj'] = data[['R_Q1', 'R_Q2', 'R_Q4', 'R_LH1', 'R_LH2']].min(axis=1)*1e3

# ROG wrt VACTH magnetic reconstruction in [mm]
data['ROG_LH'] = data['R_obj'] - data['Rext_median']
data['ROG_Q1'] = data['R_Q1']*1e3 - data['Rext_median']
data['ROG_Q2'] = data['R_Q2']*1e3 - data['Rext_median']
data['ROG_Q4'] = data['R_Q4']*1e3 - data['Rext_median']
data['ROG'] = data['R_obj'] - data['Rext_median']
# ROG wrt NICE magnetic reconstruction in [mm]
data['ROG_Q1_NICE'] = data['R_Q1']*1e3 - data['Rext_median_NICE']
data['ROG_Q2_NICE'] = data['R_Q2']*1e3 - data['Rext_median_NICE']
data['ROG_Q4_NICE'] = data['R_Q4']*1e3 - data['Rext_median_NICE']
data['ROG_NICE'] = data['R_obj'] - data['Rext_median_NICE']


In [ ]:
# saving back the data
data.to_csv('WEST_C5_database_resumed_parameters_cleaned.csv')

# Stats

In [ ]:
data['nl']

In [ ]:
data['LH1_over_LH2'] = data['LH_P_LH1']/data['LH_P_LH2']
fig, ax = plt.subplots()
data.query('Ip > 0.48 and Ip < 0.52 and LH_P_tot>0.5 and LH1_over_LH2>0 and LH1_over_LH2 < 2').plot(ax=ax, x='nl', y='LH_P_tot', kind='scatter', c='LH1_over_LH2', cmap='tab10')
ax.set_xlabel('nl')
ax.set_ylabel('Total LH Power [MW]')

# Coupling Resistances

In [ ]:
data['P_over_Rc_Q1'] = data['IC_P_Q1']/data['IC_Rc_Q1_avg']
data['P_over_Rc_Q2'] = data['IC_P_Q2']/data['IC_Rc_Q2_avg']
data['P_over_Rc_Q4'] = data['IC_P_Q4']/data['IC_Rc_Q4_avg']

_data_Q1 = data['P_over_Rc_Q1'].replace([np.inf, -np.inf], np.nan).dropna()
_data_Q1 = _data_Q1[_data_Q1>0]
_data_Q2 = data['P_over_Rc_Q2'].replace([np.inf, -np.inf], np.nan).dropna()
_data_Q2 = _data_Q2[_data_Q2>0]
_data_Q4 = data['P_over_Rc_Q4'].replace([np.inf, -np.inf], np.nan).dropna()
_data_Q4 = _data_Q4[_data_Q4>0]

In [ ]:
data.query('IC_Rc_Q1_avg>1.5 or IC_Rc_Q1_avg>1.5')['LH_P_tot'].std()

In [ ]:
fig, ax = plt.subplots()
data.plot('IC_Rc_Q1_avg', 'IC_P_Q1', kind='scatter', ax=ax, color='C0', label='Q1')
data.plot('IC_Rc_Q2_avg', 'IC_P_Q2', kind='scatter', ax=ax, color='C1', label='Q2')
#data.plot('IC_Rc_Q4_avg', 'IC_P_Q4', kind='scatter', ax=ax, color='C2', label='Q4')
ax.set_xlabel('Average Coupling Resistance [Ohm]')
ax.set_ylabel('Coupled Power/antenna [MW]')


In [ ]:
data['IC_Rc_Q1_avg'].hist(bins=100)

In [ ]:
fig, ax = plt.subplots()
data.query('IC_P_Q1 > 0.1').plot(kind='hist', ax=ax, y='IC_Rc_Q1_avg', bins=100, alpha=0.5)
data.query('IC_P_Q2 > 0.1').plot(kind='hist', ax=ax, y='IC_Rc_Q2_avg', bins=100, alpha=0.5)
data.query('IC_P_Q4 > 0.1').plot(kind='hist', ax=ax, y='IC_Rc_Q4_avg', bins=100, )
ax.set_title('WEST C5 - ICRH Coupling')
ax.set_xlabel('<Rc> [Ohm]')

In [ ]:
data

In [ ]:
pd.concat([
    data.query('IC_P_Q1 > 0.2'),
    data.query('IC_P_Q2 > 0.2'),
    data.query('IC_P_Q4 > 0.2')]).drop_duplicates()




In [ ]:
# Cumulated data
avg_Rc = np.r_[data.query('IC_P_Q1 > 0.2')['IC_Rc_Q1_avg'].values,
     data.query('IC_P_Q2 > 0.2')['IC_Rc_Q2_avg'].values,
     data.query('IC_P_Q4 > 0.2')['IC_Rc_Q4_avg'].values
     ]
avg_Rc = avg_Rc[~np.isnan(avg_Rc)]
avg_Rc = avg_Rc[avg_Rc>0.1]

In [ ]:
fig, ax = plt.subplots()
sns.histplot(avg_Rc, ax=ax, binwidth=0.05, stat='percent')
ax.set_xlabel('< $R_c$ > [$\Omega$]')
ax.set_title(f'WEST 2021 - ICRH Coupling ({len(avg_Rc)} points)')

In [ ]:
fig, ax = plt.subplots()
data.query('IC_P_Q1 > 0.1').plot(kind='hist', ax=ax, y='IC_Rc_Q1_avg', bins=100, alpha=0.5)
data.query('IC_P_Q2 > 0.1').plot(kind='hist', ax=ax, y='IC_Rc_Q2_avg', bins=100, alpha=0.5)
data.query('IC_P_Q4 > 0.1').plot(kind='hist', ax=ax, y='IC_Rc_Q4_avg', bins=100, )



# Electric Field vs Xpoint

In [ ]:
data['IC_Voltage_ratio_upper_lower_Q1_left'] = data['IC_Voltage_left_upper_Q1']/data['IC_Voltage_left_lower_Q1']
data['IC_Voltage_ratio_upper_lower_Q1_right'] = data['IC_Voltage_right_upper_Q1']/data['IC_Voltage_right_lower_Q1']

data['IC_Voltage_ratio_upper_lower_Q2_left'] = data['IC_Voltage_left_upper_Q2']/data['IC_Voltage_left_lower_Q2']
data['IC_Voltage_ratio_upper_lower_Q2_right'] = data['IC_Voltage_right_upper_Q2']/data['IC_Voltage_right_lower_Q2']


In [ ]:
_data1_LSN = data.query("IC_P_Q1 > 0.1 & Xpoint == 'LSN' & pulse>@phase_control_OK").groupby('pulse')
_data1_USN = data.query("IC_P_Q1 > 0.1 & Xpoint == 'USN' & pulse>@phase_control_OK").groupby('pulse')
_data2_LSN = data.query("IC_P_Q2 > 0.1 & Xpoint == 'LSN' & pulse>@phase_control_OK").groupby('pulse')
_data2_USN = data.query("IC_P_Q2 > 0.1 & Xpoint == 'USN' & pulse>@phase_control_OK").groupby('pulse')


fig, ax = plt.subplots()
_data1_LSN.mean().reset_index().plot(ax=ax, kind='scatter', 
                                     x='pulse', y='IC_Voltage_ratio_upper_lower_Q1_left', 
                                     yerr=_data1_LSN.std().reset_index(),
                                     marker='o', label='Q1 LSN')
_data1_USN.mean().reset_index().plot(ax=ax, kind='scatter', 
                                     x='pulse', y='IC_Voltage_ratio_upper_lower_Q1_right', 
                                     yerr=_data1_USN.std().reset_index(),
                                     marker='o', color='C1')

_data1_LSN.mean().reset_index().plot(ax=ax, kind='scatter', 
                                     x='pulse', y='IC_Voltage_ratio_upper_lower_Q1_left', 
                                     yerr=_data1_LSN.std().reset_index(),
                                     marker='^', label='Q1 USN')
_data1_USN.mean().reset_index().plot(ax=ax, kind='scatter', 
                                     x='pulse', y='IC_Voltage_ratio_upper_lower_Q1_right', 
                                     yerr=_data1_USN.std().reset_index(),
                                     marker='^', color='C1')

_data2_LSN.mean().reset_index().plot(ax=ax, kind='scatter', 
                                     x='pulse', y='IC_Voltage_ratio_upper_lower_Q2_left', 
                                     yerr=_data2_LSN.std().reset_index(),
                                     marker='o', label='Q2 LSN')
_data2_USN.mean().reset_index().plot(ax=ax, kind='scatter', 
                                     x='pulse', y='IC_Voltage_ratio_upper_lower_Q2_right', 
                                     yerr=_data2_USN.std().reset_index(),
                                     marker='o', color='C1')

_data2_LSN.mean().reset_index().plot(ax=ax, kind='scatter', 
                                     x='pulse', y='IC_Voltage_ratio_upper_lower_Q2_left', 
                                     yerr=_data2_LSN.std().reset_index(),
                                     marker='^', label='Q2 USN')
_data2_USN.mean().reset_index().plot(ax=ax, kind='scatter', 
                                     x='pulse', y='IC_Voltage_ratio_upper_lower_Q2_right', 
                                     yerr=_data2_USN.std().reset_index(),
                                     marker='^', color='C1')



In [ ]:
pulses = data.query("IC_P_Q2 > 0.1 & IC_P_Q1 < 0.1 & IC_P_Q4 <0.1 & LH_P_tot < 0.1 & pulse > @phase_control_OK")['pulse'].unique()

In [ ]:
for pulse in pulses:
    print(pulse, data.query('pulse == @pulse').groupby('pulse').max()['IC_P_Q2'].values)
    

# ROG Evolution

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
_data = data.query("P_RF>0.5 & MHD < 50 & Xpoint == 'LSN'").groupby('pulse')
_data.mean().reset_index().plot(ax=ax, kind='scatter',  yerr=_data.std().reset_index(),
                                x='pulse', y='ROG', label='VACTH LSN', 
                                color='C0', alpha=0.5,  s=80)
_data.mean().reset_index().plot(ax=ax, kind='scatter', yerr=_data.std().reset_index(),
                                x='pulse', y='ROG_NICE', label='NICE LSN', 
                                color='C1', alpha=0.5, s=80)
_data = data.query("P_RF>0.5 & MHD < 50 & Xpoint == 'USN'").groupby('pulse')
_data.mean().reset_index().plot(ax=ax, kind='scatter',  yerr=_data.std().reset_index(),
                                x='pulse', y='ROG', label='VACTH USN', marker='^', edgecolor='k',
                                color='C0', alpha=0.5,  s=80)
_data.mean().reset_index().plot(ax=ax, kind='scatter', yerr=_data.std().reset_index(),
                                x='pulse', y='ROG_NICE', label='NICE USN', marker='^', edgecolor='k',
                                color='C1', alpha=0.5, s=80)

ax.set_xlabel('WEST Pulse #')
ax.set_ylabel('ROG [mm]')
ax.set_ylim(0, 75)
ax.set_title('Data: pulse averaged, times for RF>0.5 MW ; MHD < 50')
ax.legend(ncol=2)

# Phase feedback Evolution

In [ ]:
fig, ax = plt.subplots()
data.query('IC_P_Q1>0.05').plot(ax=ax, kind='scatter', 
                                x='pulse', y='IC_Phase_Q1', 
                                label='Q1', color='C0',  s=80, alpha=.5, edgecolor='k')
data.query('IC_P_Q2>0.05').plot(ax=ax, kind='scatter', 
                                x='pulse', y='IC_Phase_Q2', 
                                label='Q2', color='C1',  s=80, alpha=.5, edgecolor='k')
data.query('IC_P_Q4>0.05').plot(ax=ax, kind='scatter', 
                                x='pulse', y='IC_Phase_Q4', 
                                label='Q4', color='C2',  s=80, alpha=.5, edgecolor='k')
ax.annotate('#56469: Phase Control OK', xy=(56469, 190), xytext=(56500, 250), 
            arrowprops=dict(facecolor='black'), zorder=1)
ax.axhline(180, ls='--', color='gray', zorder=0)
ax.set_ylabel('Toroidal Phase [deg]')
ax.set_xlabel('WEST Pulse #')
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
data_Q1=data.query('IC_P_Q1>0.05').groupby('pulse')
data_Q2=data.query('IC_P_Q2>0.05').groupby('pulse')
data_Q4=data.query('IC_P_Q4>0.05').groupby('pulse')

data_Q1.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_Phase_Q1', label='Q1', color='C0', yerr=data_Q1.std().reset_index(), s=80, alpha=.8)
data_Q2.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_Phase_Q2', label='Q2', color='C1', yerr=data_Q2.std().reset_index(), s=80, alpha=.8)
data_Q4.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_Phase_Q4', label='Q4', color='C2', yerr=data_Q4.std().reset_index(), s=80, alpha=.8)
ax.annotate('#56469: Phase Control OK', xy=(56469, 190), xytext=(56500, 250), 
            arrowprops=dict(facecolor='black'), zorder=1)
ax.axhline(180, ls='--', color='gray', zorder=0)
ax.set_title('Pulse Averaged Data')
ax.set_ylabel('Toroidal Phase [deg]')
ax.set_xlabel('WEST Pulse #')

# Coupling Resistance vs...

In [ ]:
fig, ax = plt.subplots()
_data = pd.concat([data.query("IC_P_Q1 > 0.1 & pulse>=@phase_control_OK  "), 
                  data.query("IC_P_Q1 > 0.1 & pulse>=@phase_control_OK "), 
                  data.query("IC_P_Q2 > 0.1 & pulse>=@phase_control_OK & pulse>=@Q2_position_OK "),
                  data.query("IC_P_Q2 > 0.1 & pulse>=@phase_control_OK & pulse>=@Q2_position_OK "),
                  data.query("IC_P_Q4 > 0.1 & pulse>=@phase_control_OK "),
                  data.query("IC_P_Q4 > 0.1 & pulse>=@phase_control_OK ")])

In [ ]:
_data

In [ ]:
fig, ax = plt.subplots()
data.query("IC_P_Q1 > 0.1 & pulse>=@phase_control_OK ").plot(ax=ax, kind='scatter', x='ROG_NICE', y='IC_Rc_Q1_left', color='C0', label='Q1', alpha=0.4)
data.query("IC_P_Q1 > 0.1 & pulse>=@phase_control_OK ").plot(ax=ax, kind='scatter', x='ROG_NICE', y='IC_Rc_Q1_right', color='C0', alpha=0.4)

data.query("IC_P_Q2 > 0.1 & pulse>=@phase_control_OK & pulse>=@Q2_position_OK ").plot(ax=ax, kind='scatter', x='ROG_NICE', y='IC_Rc_Q2_left', color='C1', label='Q2', alpha=0.4)
data.query("IC_P_Q2 > 0.1 & pulse>=@phase_control_OK & pulse>=@Q2_position_OK ").plot(ax=ax, kind='scatter', x='ROG_NICE', y='IC_Rc_Q2_right', color='C1', alpha=0.4)

data.query("IC_P_Q4 > 0.1 & pulse>=@phase_control_OK ").plot(ax=ax, kind='scatter', x='ROG_NICE', y='IC_Rc_Q4_left', color='C2', label='Q4', alpha=0.4)
data.query("IC_P_Q4 > 0.1 & pulse>=@phase_control_OK ").plot(ax=ax, kind='scatter', x='ROG_NICE', y='IC_Rc_Q4_right', color='C2', alpha=0.4)

ax.set_xlim(left=0, right=80)
ax.set_ylabel('Coupling Resistance Rc [Ohm]')
ax.set_title('Data: P_IC>0.1 NICE')
#ax.scatter(x, y, color='k')

In [ ]:

fig, ax = plt.subplots()
data.query("IC_P_Q1 > 0.1 & pulse>=@phase_control_OK  ").plot(ax=ax, kind='scatter', x='ROG', y='IC_Rc_Q1_left', color='C0', label='Q1', alpha=0.4)
data.query("IC_P_Q1 > 0.1 & pulse>=@phase_control_OK ").plot(ax=ax, kind='scatter', x='ROG', y='IC_Rc_Q1_right', color='C0', alpha=0.4)

data.query("IC_P_Q2 > 0.1 & pulse>=@phase_control_OK & pulse>=@Q2_position_OK ").plot(ax=ax, kind='scatter', x='ROG', y='IC_Rc_Q2_left', color='C1', label='Q2', alpha=0.4)
data.query("IC_P_Q2 > 0.1 & pulse>=@phase_control_OK & pulse>=@Q2_position_OK ").plot(ax=ax, kind='scatter', x='ROG', y='IC_Rc_Q2_right', color='C1', alpha=0.4)

data.query("IC_P_Q4 > 0.1 & pulse>=@phase_control_OK ").plot(ax=ax, kind='scatter', x='ROG', y='IC_Rc_Q4_left', color='C2', label='Q4', alpha=0.4)
data.query("IC_P_Q4 > 0.1 & pulse>=@phase_control_OK ").plot(ax=ax, kind='scatter', x='ROG', y='IC_Rc_Q4_right', color='C2', alpha=0.4)

ax.set_xlim(left=0, right=30)
ax.set_ylabel('Coupling Resistance Rc [Ohm]')
ax.set_title('Data: P_IC>0.1 VACTH')

# Radiated Power Evolution

In [ ]:
fig, ax = plt.subplots()
datagrp_LSN = data.query("Prad_imas>0 & P_RF > 0.5 & pulse > @post_boro[1] & Xpoint == 'LSN'").groupby('pulse')
datagrp_USN = data.query("Prad_imas>0 & P_RF > 0.5 & pulse > @post_boro[1] & Xpoint == 'USN'").groupby('pulse')

cbar=datagrp_LSN.mean().reset_index().plot(ax=ax, kind='scatter', x='Ptot', y='Prad_imas', alpha=0.6, edgecolor='k',
                               c='ratio_Pic_Plh', cmap='plasma', yerr=datagrp_LSN.std(), s=60, vmin=0, vmax=1)
datagrp_USN.mean().reset_index().plot(ax=ax, kind='scatter', x='Ptot', y='Prad_imas', alpha=0.6, edgecolor='k',
                               c='ratio_Pic_Plh', cmap='plasma', yerr=datagrp_USN.std(), s=60, marker='^', colorbar=False, vmin=0, vmax=1)
ax.legend(('LSN', 'USN'))
ax.plot([0,7], [0, 0.5*7], ls='--', color='k', zorder=-10)
ax.plot([0,7], [0, 0.4*7], ls='--', color='grey', zorder=-10)
ax.plot([0,7], [0, 0.6*7], ls='--', color='grey', zorder=-10)
ax.set_xlim(left=0, right=6)
ax.set_ylim(bottom=0, top=4)
ax.set_title('P_RF > 0.5')
ax.text(5.5, 2.3, '40%', rotation=30)
ax.text(5.5, 2.9, '50%', rotation=35)
ax.text(5.5, 3.4, '60%', rotation=40)

ax.set_ylabel('Radiated Power [MW]')
ax.set_xlabel('Total Power (Ohmic+RF) [MW]')
ax.set_title('WEST - January 2021 (pulse average for RF>0.5MW)')
cbar.set_label('$P_{ic}/P_{lh}$')

In [ ]:
data['frad_tot']= data['Prad_imas']/data['Ptot']

#_data_C4_LSN = data_C4.query("Prad_imas>0 & P_RF > 0.5 & pulse > @post_boro[1] & Xpoint == 'LSN'")
#_data_C4_USN = data_C4.query("Prad_imas>0 & P_RF > 0.5 & pulse > @post_boro[1] & Xpoint == 'USN'")
_data_C5_LSN = data.query("Prad_imas>0 & P_RF > 0.5 & pulse > @post_boro[1] & Xpoint == 'LSN'").groupby('pulse')
_data_C5_USN = data.query("Prad_imas>0 & P_RF > 0.5 & pulse > @post_boro[1] & Xpoint == 'USN'").groupby('pulse')


fig, ax = plt.subplots()
_data_C5_LSN.mean().plot(ax=ax, kind='scatter', x='ratio_Pic_Plh', y='frad_tot', s=60, xerr=_data_C5_LSN.std(), yerr=_data_C5_LSN.std())
_data_C5_USN.mean().plot(ax=ax, kind='scatter', x='ratio_Pic_Plh', y='frad_tot', marker='^', s=60, xerr=_data_C5_USN.std(), yerr=_data_C5_USN.std())

ax.set_xlim(left=0, right=1)

# Coupling Resistance Evolution

In [ ]:
# means and errors values for each pulse
data_means = data.query('IC_P_tot > 0.2').groupby('pulse').mean().reset_index()
data_errors = data.query('IC_P_tot > 0.2').groupby('pulse').std().reset_index()

fig, ax = plt.subplots()

data_means.plot(kind='scatter', ax=ax, x='pulse_relative', y='IC_Q1_Rc_left_over_right', label='Q1', yerr=data_errors, color='C0')
data_means.plot(kind='scatter', ax=ax, x='pulse_relative', y='IC_Q2_Rc_left_over_right', label='Q2', yerr=data_errors, color='C1')
data_means.plot(kind='scatter', ax=ax, x='pulse_relative', y='IC_Q4_Rc_left_over_right', label='Q4', yerr=data_errors, color='C2')

ax.set_ylim(bottom=0, top=2.5)
ax.set_ylabel('Rc left / Rc right')


In [ ]:
fig, ax = plt.subplots()

data.plot(kind='scatter', ax=ax, x='pulse', y='IC_Q1_Rc_left_over_right', label='Q1', color='C0', alpha=0.5)
data.plot(kind='scatter', ax=ax, x='pulse', y='IC_Q2_Rc_left_over_right', label='Q2', color='C1', alpha=0.5)
data.plot(kind='scatter', ax=ax, x='pulse', y='IC_Q4_Rc_left_over_right', label='Q4', color='C2', alpha=0.5)

ax.set_ylim(bottom=0, top=2.5)
ax.set_ylabel('Rc left / Rc right')

In [ ]:
fig, ax = plt.subplots(3,1,sharex=True)
sns.violinplot(ax=ax[0], x='IC_Q1_Rc_left_over_right', data=data.query('IC_P_Q1 > 0.2 & pulse >= @phase_control_OK'), color='C0', inner=None, label='Q1')
sns.violinplot(ax=ax[1], x='IC_Q2_Rc_left_over_right', data=data.query('IC_P_Q2 > 0.2 & pulse >= @phase_control_OK & pulse < @Q2_position_OK'), color='C1', inner=None, label='Q2')
sns.violinplot(ax=ax[1], x='IC_Q2_Rc_left_over_right', data=data.query('IC_P_Q2 > 0.2 & pulse >= @phase_control_OK & pulse >= @Q2_position_OK'), color='orange', inner=None, label='Q2 Position Corrected', alpha=0.1)
sns.violinplot(ax=ax[2], x='IC_Q4_Rc_left_over_right', data=data.query('IC_P_Q4 > 0.2'), color='C2', inner=None, label='Q4')
ax[0].set_xlim(0, 2) 
fig.subplots_adjust(hspace=0)
ax[0].set_xlabel(''); ax[1].set_ylabel('')
ax[-1].set_xlabel('Rc left / Rc right')
[a.legend([f'Q{ant}']) for a,ant in zip(ax, ['1', '2', '4'])]
[a.axvline(1, ls='--', color='gray') for a in ax]

# Impurity Content Evolution

In [ ]:
# means and errors values for each pulse
data_means = data.query('P_RF>0.2').groupby('pulse').mean().reset_index()
data_errors = data.query('P_RF>0.2').groupby('pulse').std().reset_index()

fig, ax = plt.subplots(3, 1, sharex=True)

data_means.plot(ax=ax[0], kind='scatter', 
                x='pulse_relative', y='Cu_norm', yerr=data_errors,
               color='darkorange', label='Cu/nl', s=60, edgecolor='k', alpha=0.7)

data_means.plot(ax=ax[1], kind='scatter', 
                x='pulse_relative', y='Fe_norm', yerr=data_errors,
               color='lightgray', label='Fe/nl', s=60, edgecolor='k', alpha=0.7)
data_means.plot(ax=ax[2], kind='scatter', 
                x='pulse_relative', y='Ag19_norm', yerr=data_errors,
               color='lightblue', label='Ag19/nl', s=60, edgecolor='k', alpha=0.7)


ax[0].set_title('$P_{RF}$ > 0.2 MW')
[a.set_ylim(bottom=0) for a in ax]

[[a.axvline(shot-first_C5_pulse, ls='--', color='gray') for shot in post_boro] for a in ax]
[a.axvline(january_restart-first_C5_pulse, ls='-.', color='gray') for a in ax]
ax[0].annotate('Christmas Break', xy=(56544-first_C5_pulse, 300), xytext=(200, 350), 
            arrowprops=dict(facecolor='black'))

ax[0].annotate('Boro', xy=(56597-first_C5_pulse, 300), xytext=(400, 350), 
            arrowprops=dict(facecolor='black'))
ax[0].set_ylim(0,300)
ax[1].set_ylim(0,30)
ax[2].set_ylim(0,70)

[a.set_ylabel('a.u.') for a in ax]
# liste des chocs avec puissance FCI
IC_pulses = data.query('IC_P_tot > 0.2')['pulse'].unique()
for IC_pulse in IC_pulses:
    [a.axvline(IC_pulse-first_C5_pulse, ls='-', color='gray', alpha=0.2) for a in ax]

# Isotopic Ratio vs ...

In [ ]:
fig, ax = plt.subplots()
# Boros : 56247, 56641
data_LH_only=data.query("LH_P_tot > 0.3 & IC_P_tot < 0.05 & Cu_norm > 0 & pulse > @january_restart & Xpoint == 'USN'").groupby('pulse')
data_IC_only=data.query("LH_P_tot < 0.3 & IC_P_tot > 0.3 & Cu_norm > 0 & pulse > @january_restart & Xpoint == 'USN'").groupby('pulse')
data_LH_and_IC=data.query("LH_P_tot > 0.3 & IC_P_tot > 0.3 & Cu_norm > 0 & pulse > @january_restart & Xpoint == 'USN'").groupby('pulse')

data_LH_only.mean().plot(ax=ax, kind='scatter',
            x='P_RF', y='Cu_norm', yerr=data_LH_only.std(),
            cmap='plasma', s=40, alpha=0.6, label='C5 experiments LH only', colorbar=False)

data_LH_and_IC.mean().plot(ax=ax, kind='scatter',
            x='P_RF', y='Cu_norm', color='C1', yerr=data_LH_and_IC.std(),
            cmap='plasma', alpha=0.6, s=100, marker='d', label='C5 experiments LH+IC', colorbar=False, edgecolor='k')

data_IC_only.mean().plot(ax=ax, kind='scatter',
            x='P_RF', y='Cu_norm', color='C2', yerr=data_IC_only.std(),
            cmap='plasma', alpha=0.6, s=100, marker='o', label='C5 experiments IC only', colorbar=False, edgecolor='k')

# ax.set_xscale('log')
ax.set_title('pulse average / USN')
ax.set_ylim(bottom=0, top=400)
ax.set_ylabel('Cu/nl [a.u.]')
ax.set_xlabel('RF Power [MW]')

In [ ]:
fig, ax = plt.subplots()
# Boros : 56247, 56641
data_LH_only=data.query("LH_P_tot > 0.3 & IC_P_tot < 0.05 & Cu_norm > 0 & pulse > @january_restart & Xpoint == 'LSN'").groupby('pulse')
data_IC_only=data.query("LH_P_tot < 0.3 & IC_P_tot > 0.3 & Cu_norm > 0 & pulse > @january_restart & Xpoint == 'LSN'").groupby('pulse')
data_LH_and_IC=data.query("LH_P_tot > 0.3 & IC_P_tot > 0.3 & Cu_norm > 0 & pulse > @january_restart & Xpoint == 'LSN'").groupby('pulse')

data_LH_only.mean().plot(ax=ax, kind='scatter',
            x='P_RF', y='Cu_norm', yerr=data_LH_only.std(),
            cmap='plasma', s=40, alpha=0.6, label='C5 experiments LH only', colorbar=False)

data_LH_and_IC.mean().plot(ax=ax, kind='scatter',
            x='P_RF', y='Cu_norm', color='C1', yerr=data_LH_and_IC.std(),
            cmap='plasma', alpha=0.6, s=100, marker='d', label='C5 experiments LH+IC', colorbar=False, edgecolor='k')

data_IC_only.mean().plot(ax=ax, kind='scatter',
            x='P_RF', y='Cu_norm', color='C2', yerr=data_IC_only.std(),
            cmap='plasma', alpha=0.6, s=100, marker='o', label='C5 experiments IC only', colorbar=False, edgecolor='k')

# ax.set_xscale('log')
ax.set_title('pulse average / LSN')
ax.set_ylim(bottom=0, top=400)
ax.set_ylabel('Cu/nl [a.u.]')
ax.set_xlabel('RF Power [MW]')

In [ ]:
data_LH_and_IC.mean().reset_index().query('Cu_norm < 170 & P_RF > 3.5')['pulse']

In [ ]:
fig, ax = plt.subplots(1,2,sharey=True)
# Boros : 56247, 56641
data.query(f'IC_P_tot > 0.3 & Cu_norm > 10  & `Isotopic Ratio INBUM04`>0 & MHD < 50').plot(ax=ax[0], kind='scatter', 
                                x='Isotopic Ratio INBUM04', y='Cu_norm', 
                                c='LH_P_tot', cmap='plasma', alpha=0.8, s=40, label='C5 experiments with ICRH', colorbar=False)
data.query(f'IC_P_tot > 0.3 & Cu_norm > 10  & `Isotopic Ratio LODIVOU15`>0 & MHD < 50').plot(ax=ax[1], kind='scatter', 
                                x='Isotopic Ratio LODIVOU15', y='Cu_norm', 
                                c='LH_P_tot', cmap='plasma', alpha=0.8, s=40, label='C5 experiments with ICRH', colorbar=False)
im = plt.gca().get_children()[0]
cax = fig.add_axes([0.92,0.1,0.02,0.8]) 
fig.colorbar(im, cax=cax, label='LH Power [MW]')
fig.suptitle('data for: $P_{IC}>0.3$MW ; MHD < 50')
[a.set_xlim(0, 25) for a in ax]
[a.set_ylim(bottom=0, top=500) for a in ax]
[a.legend(loc='upper center') for a in ax]
[a.set_ylabel('$Cu/n_l$ [a.u.]') for a in ax]
fig.subplots_adjust(wspace=0.05, left=.08)
#fig.tight_layout()


In [ ]:
fig, ax = plt.subplots(1,2,sharey=True,sharex=True)
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data_means = data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse > 56450 & `Isotopic Ratio INBUM04`>0').groupby('pulse').mean().reset_index()
data_errors = data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse > 56450 &  `Isotopic Ratio INBUM04`>0').groupby('pulse').std().reset_index()

data_means.plot(ax=ax[0], kind='scatter', 
                                x='Isotopic Ratio INBUM04', y='Cu_norm', yerr=data_errors,
                                c='ratio_Pic_Plh', cmap='plasma', alpha=0.8,  s=50, label='Before Chrismas Break', colorbar=False, vmin=0, vmax=1)
data_means.plot(ax=ax[1], kind='scatter', 
                                x='Isotopic Ratio LODIVOU15', y='Cu_norm', yerr=data_errors,
                                c='ratio_Pic_Plh', cmap='plasma', alpha=0.8, s=50, label='Before Chrismas Break', colorbar=False, vmin=0, vmax=1)
im = plt.gca().get_children()[0]
cax = fig.add_axes([0.91,0.1,0.02,0.88]) 
fig.colorbar(im, cax=cax, label='$P_{IC}/P_{LH}$')
fig.suptitle('data for $P_{RF} > 0.3$ MW ; size proportional to $P_{IC}$')
[a.set_xlim(0, 30) for a in ax]
[a.set_ylim(bottom=0, top=450) for a in ax]
[a.legend(loc='upper center') for a in ax]
[a.set_ylabel('$Cu/n_l$') for a in ax]
fig.subplots_adjust(wspace=0.05, left=0.08, top=0.92)
#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,2,sharey=True,sharex=True)
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data_means = data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse < {january_restart} & `Isotopic Ratio INBUM04`>0').groupby('pulse').mean().reset_index()
data_errors = data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse < {january_restart} & `Isotopic Ratio INBUM04`>0').groupby('pulse').std().reset_index()

data_means.plot(ax=ax[0], kind='scatter', 
                                x='Isotopic Ratio INBUM04', y='Cu_norm', yerr=data_errors,
                                c='P_RF', cmap='plasma', alpha=0.8,  s=(60+1000*data_means.replace([np.inf, -np.inf], 0)['IC_P_tot']**2), label='Before Chrismas Break', colorbar=False)
data_means.plot(ax=ax[1], kind='scatter', 
                                x='Isotopic Ratio LODIVOU15', y='Cu_norm', yerr=data_errors,
                                c='P_RF', cmap='plasma', alpha=0.8, s=(60+1000*data_means.replace([np.inf, -np.inf], 0)['IC_P_tot']**2), label='Before Chrismas Break', colorbar=False)
im = plt.gca().get_children()[0]
cax = fig.add_axes([0.91,0.1,0.02,0.88]) 
fig.colorbar(im, cax=cax, label='$P_{RF}$ [MW]')
fig.suptitle('data for $P_{RF} > 0.3$ MW ; size proportional to $P_{IC}$')
[a.set_xlim(0, 30) for a in ax]
[a.set_ylim(bottom=0, top=450) for a in ax]
[a.legend(loc='upper center') for a in ax]
[a.set_ylabel('$Cu/n_l$') for a in ax]
fig.subplots_adjust(wspace=0.05, left=0.08, top=0.92)
#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,2,sharey=True,sharex=True)
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data_means = data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse > {post_boro[1]} & `Isotopic Ratio INBUM04`>0').groupby('pulse').mean().reset_index()
data_errors = data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse > {post_boro[1]} & `Isotopic Ratio INBUM04`>0').groupby('pulse').std().reset_index()

data_means.plot(ax=ax[0], kind='scatter', 
                                x='Isotopic Ratio INBUM04', y='Cu_norm', yerr=data_errors,
                                c='P_RF', cmap='plasma', alpha=0.8,  s=(60+100*data_means.replace([np.inf, -np.inf], 0)['IC_P_tot']**2), label='After Chrismas Break', colorbar=False)
data_means.plot(ax=ax[1], kind='scatter', 
                                x='Isotopic Ratio LODIVOU15', y='Cu_norm', yerr=data_errors,
                                c='P_RF', cmap='plasma', alpha=0.8, s=(60+100*data_means.replace([np.inf, -np.inf], 0)['IC_P_tot']**2), label='After Chrismas Break', colorbar=False)
im = plt.gca().get_children()[0]
cax = fig.add_axes([0.91,0.1,0.02,0.88]) 
fig.colorbar(im, cax=cax, label='$P_{RF}$ [MW]')
fig.suptitle('data for $P_{RF} > 0.3$ MW ; size proportional to $P_{IC}/P_{LH}$')
[a.set_xlim(0, 12) for a in ax]
[a.set_ylim(bottom=0, top=450) for a in ax]
[a.legend(loc='upper center') for a in ax]
[a.set_ylabel('$Cu/n_l$') for a in ax]
fig.subplots_adjust(wspace=0.05, left=0.08, top=0.92)
#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,2,sharey=True,sharex=True)
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data_means = data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse < {january_restart} & `Isotopic Ratio INBUM04`>0').groupby('pulse').mean().reset_index()
data_errors = data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse < {january_restart} & `Isotopic Ratio INBUM04`>0').groupby('pulse').std().reset_index()

data_means.plot(ax=ax[0], kind='scatter', 
                                x='Isotopic Ratio INBUM04', y='Cu_norm_nl_RF', yerr=data_errors,
                                c='P_RF', cmap='plasma', alpha=0.8, label='Before Chrismas Break', colorbar=False, s=(60+1000*data_means['IC_P_tot']**2))
data_means.plot(ax=ax[1], kind='scatter', 
                                x='Isotopic Ratio LODIVOU15', y='Cu_norm_nl_RF', yerr=data_errors,
                                c='P_RF', cmap='plasma', alpha=0.8,  label='Before Chrismas Break', colorbar=False, s=(60+1000*data_means['IC_P_tot']**2))
im = plt.gca().get_children()[0]
cax = fig.add_axes([0.91,0.1,0.02,0.88]) 
fig.colorbar(im, cax=cax)
fig.suptitle('$P_{RF} > 0.3$ MW ; colorbar: P_RF ; size: P_IC')
[a.set_xlim(0, 30) for a in ax]
[a.set_ylim(bottom=0, top=200) for a in ax]
[a.legend(loc='upper center') for a in ax]
[a.set_ylabel('$Cu/n_l/P_{RF}$ [a.u.]') for a in ax]
fig.subplots_adjust(wspace=0.05, left=0.08, top=0.92)
#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,2,sharey=True,sharex=True)
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data_means = data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse > {january_restart} & `Isotopic Ratio INBUM04`>0').groupby('pulse').mean().reset_index()
data_errors = data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse > {january_restart} & `Isotopic Ratio INBUM04`>0').groupby('pulse').std().reset_index()

data_means.plot(ax=ax[0], kind='scatter', 
                                x='Isotopic Ratio INBUM04', y='Cu_norm_nl_RF', yerr=data_errors,
                                c='P_RF', cmap='plasma', alpha=0.8, label='After Chrismas Break', colorbar=False, s=(60+100*data_means['IC_P_tot']**2))
data_means.plot(ax=ax[1], kind='scatter', 
                                x='Isotopic Ratio LODIVOU15', y='Cu_norm_nl_RF', yerr=data_errors,
                                c='P_RF', cmap='plasma', alpha=0.8,  label='After Chrismas Break', colorbar=False, s=(60+100*data_means['IC_P_tot']**2))
im = plt.gca().get_children()[0]
cax = fig.add_axes([0.91,0.1,0.02,0.88]) 
fig.colorbar(im, cax=cax)
fig.suptitle('$P_{RF} > 0.3$ MW ; colorbar: P_RF ; size: P_IC')
[a.set_xlim(0, 20) for a in ax]
[a.set_ylim(bottom=0, top=200) for a in ax]
[a.legend(loc='upper center') for a in ax]
[a.set_ylabel('$Cu/n_l/P_{RF}$ [a.u.]') for a in ax]
fig.subplots_adjust(wspace=0.05, left=0.08, top=0.92)
#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,2,sharey=True,sharex=True)
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse < {january_restart} & `Isotopic Ratio INBUM04`>0').plot(ax=ax[0], kind='scatter', 
                                x='Isotopic Ratio INBUM04', y='Cu_norm_nl_IC', 
                                c='pulse', cmap='plasma', alpha=0.8, s=40, label='Before Chrismas Break', colorbar=False)
data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse < {january_restart} & `Isotopic Ratio INBUM04`>0').plot(ax=ax[1], kind='scatter', 
                                x='Isotopic Ratio LODIVOU15', y='Cu_norm_nl_IC', 
                                c='pulse', cmap='plasma', alpha=0.8, s=40, label='Before Chrismas Break', colorbar=False)
im = plt.gca().get_children()[0]
cax = fig.add_axes([0.91,0.1,0.02,0.88]) 
fig.colorbar(im, cax=cax)
fig.suptitle('$P_{RF} > 0.3$ MW')
[a.set_xlim(0, 30) for a in ax]
[a.set_ylim(bottom=0, top=450) for a in ax]
[a.legend(loc='upper center') for a in ax]
[a.set_ylabel('$Cu/n_l/P_{IC}$') for a in ax]
fig.subplots_adjust(wspace=0.05, left=0.08, top=0.92)
#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,2,sharey=True,sharex=True)
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse > {post_boro[1]} & `Isotopic Ratio INBUM04`>0').plot(ax=ax[0], kind='scatter', 
                                x='Isotopic Ratio INBUM04', y='Cu_norm_nl_IC', 
                                c='pulse', cmap='plasma', alpha=0.8, s=40, label='After Boro', colorbar=False)
data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse > {post_boro[1]} & `Isotopic Ratio INBUM04`>0').plot(ax=ax[1], kind='scatter', 
                                x='Isotopic Ratio LODIVOU15', y='Cu_norm_nl_IC', 
                                c='pulse', cmap='plasma', alpha=0.8, s=40, label='After Boro', colorbar=False)
im = plt.gca().get_children()[0]
cax = fig.add_axes([0.91,0.1,0.02,0.88]) 
fig.colorbar(im, cax=cax)
fig.suptitle('$P_{RF} > 0.3$ MW')
[a.set_xlim(0, 20) for a in ax]
[a.set_ylim(bottom=0, top=450) for a in ax]
[a.legend(loc='upper center') for a in ax]
[a.set_ylabel('$Cu/n_l/P_{IC}$') for a in ax]
fig.subplots_adjust(wspace=0.05, left=0.08, top=0.92)
#fig.tight_layout()

In [ ]:
data_means = data.query('P_RF > 0.3 & Cu_norm > 10').groupby('pulse').mean().reset_index()
data_errors = data.query('P_RF > 0.3 & Cu_norm > 10').groupby('pulse').std().reset_index()

In [ ]:
fig, ax = plt.subplots()
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse < {january_restart}').plot(ax=ax, kind='scatter', 
                                y='Cu_norm', x='P_RF', 
                                c='ratio_Pic_Plh', cmap='plasma', alpha=0.8, s=40, label='Before Chrismas Break', colorbar=False, vmin=0, vmax=1)
m = plt.gca().get_children()[0]
cax = fig.add_axes([0.92,0.1,0.02,0.8]) 
fig.colorbar(im, cax=cax)

ax.set_xlim(0, 6)
ax.legend(loc='upper center')
ax.set_ylabel('$Cu/n_l$')
ax.set_xlabel('$P_{RF}$ [MW]')
fig.suptitle('$P_{RF}$>0.3MW - Color: $P_{IC}/P_{LH}$')
fig.subplots_adjust(wspace=0.05)

#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data_means.query(f'pulse < {january_restart}').plot(ax=ax, kind='scatter', 
                                y='Cu_norm', x='P_RF', yerr=data_errors,
                                c='ratio_Pic_Plh', cmap='plasma', alpha=0.8, s=40, label='Before Chrismas Break', colorbar=False, vmin=0, vmax=1)
m = plt.gca().get_children()[0]
cax = fig.add_axes([0.92,0.1,0.02,0.8]) 
fig.colorbar(im, cax=cax)

ax.set_xlim(0, 6)
ax.legend(loc='upper center')
ax.set_ylabel('$Cu/n_l$')
ax.set_xlabel('$P_{RF}$ [MW]')
fig.suptitle('$P_{RF}$>0.3MW - Color: $P_{IC}/P_{LH}$ - pulse average')
fig.subplots_adjust(wspace=0.05)

#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data.query(f'P_RF > 0.3 & Cu_norm > 10 & pulse > {post_boro[1]}').plot(ax=ax, kind='scatter', 
                                y='Cu_norm', x='P_RF', 
                                c='ratio_Pic_Plh', cmap='plasma', alpha=0.8, s=40, label='After Boro', colorbar=False, vmin=0, vmax=1)
m = plt.gca().get_children()[0]
cax = fig.add_axes([0.92,0.1,0.02,0.8]) 
fig.colorbar(im, cax=cax)

ax.set_xlim(0, 6)
ax.legend(loc='upper center')
ax.set_ylabel('$Cu/n_l$')
ax.set_xlabel('$P_{RF}$ [MW]')
fig.suptitle('$P_{RF}$>0.3MW - Color: $P_{IC}/P_{LH}$')
fig.subplots_adjust(wspace=0.05)

#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
# INBUM04 ou LODIVOU15

# Boros : 56247, 56641
data_means.query(f'pulse > {post_boro[1]}').plot(ax=ax, kind='scatter', 
                                y='Cu_norm', x='P_RF', yerr=data_errors,
                                c='ratio_Pic_Plh', cmap='plasma', alpha=0.8, s=40, label='After Boro', colorbar=False, vmin=0, vmax=1)
m = plt.gca().get_children()[0]
cax = fig.add_axes([0.92,0.1,0.02,0.8]) 
fig.colorbar(im, cax=cax)

ax.set_xlim(0, 6)
ax.legend(loc='upper center')
ax.set_ylabel('$Cu/n_l$')
ax.set_xlabel('$P_{RF}$ [MW]')
fig.suptitle('$P_{RF}$>0.3MW - Color: $P_{IC}/P_{LH} - pulse average $')
fig.subplots_adjust(wspace=0.05)

#fig.tight_layout()

# Evolution du rapport isotopique

In [ ]:
fig, ax = plt.subplots()
data_means = data.query('Ip>0.4').groupby('pulse').mean().reset_index()
data_errors = data.query('Ip>0.4').groupby('pulse').std().reset_index()

data_means.query('`Isotopic Ratio LODIVOU15`>0.5').plot(ax=ax, kind='scatter', 
                x='pulse', y='Isotopic Ratio LODIVOU15', yerr=data_errors,
               color='C0', label='Outer Divertor (LODIVOU15)', s=60, edgecolor='k', alpha=.8)
# data_means.query('`Isotopic Ratio INBUM04`>0.5').plot(ax=ax, kind='scatter', 
#                 x='pulse', y='Isotopic Ratio INBUM04', yerr=data_errors,
#                color='C1', label='Inner Bumper (INBUM04)', s=60, edgecolor='k', alpha=.8)
ax.set_xlim(left=56240, right=56925)

ax.axvline(january_restart, ls='-.', color='gray')

[ax.axvline(shot, ls='--', color='gray') for shot in post_boro]

ax.set_title('Data for $I_p>0.4$MA')
ax.set_ylim(bottom=0, top=50)
ax.set_ylabel('Isotopic Ratio [%]')
# ax.annotate('Christmas Break', xy=(56544, 32), xytext=(56400, 35), arrowprops=dict(facecolor='black'))
# ax.annotate('1st Boronization', xy=(post_boro[0], 15), xytext=(56300, 20), arrowprops=dict(facecolor='black'))
# ax.annotate('2nd Boronization', xy=(post_boro[1], 20), xytext=(56650, 25), arrowprops=dict(facecolor='black'))

ax.axhspan(5,10, color='green', alpha=0.2)
fig.tight_layout()

# Fitting to Power Scaling Laws

In [ ]:
data.columns

In [ ]:
from sklearn import linear_model

In [ ]:
# let's first filter only the usefull data from the whole dataset
signames = ['Cu_norm','IC_P_Q1', 'IC_P_Q2', 'IC_P_Q4', 'LH_P_tot']
query = f' IC_P_tot > 0.3 & Cu_norm > 10'
# Convert into log values and remove spurious points
data_log = data.query(query)[signames].apply(np.log).replace([np.inf, -np.inf], np.nan).dropna()

# Make a linear regression on the log values
lin_reg = linear_model.LinearRegression()
lin_reg.fit(data_log[signames[1:]], data_log[signames[0]])
coefs = np.squeeze(lin_reg.coef_)

print(f'Scaling coefficients: a={coefs}')
print(f'Intersection A: {lin_reg.intercept_}')

In [ ]:
def power_law(signames, coefs, A0, query='index == index or index != index'):
    y = A0
    for (signame, coef) in zip(signames[1:], coefs):
        y *= data.query(query)[signame]**coef
    return y

In [ ]:
Cu_scale = power_law(signames, coefs, np.exp(lin_reg.intercept_), query=query)

fig, ax = plt.subplots()
ax.scatter(Cu_scale, data.query(query)['Cu_norm'], alpha=0.2)
ax.plot(Cu_scale, Cu_scale, color='r')
# ax.set_xscale('log')
# ax.set_yscale('log')
ax.grid(True, which='minor')
ax.set_xlabel(r'$ \times $'.join([f'({signame})^({coef:.1f})' for (signame, coef) in zip(signames[1:], coefs)]))
ax.set_ylabel('Cu_norm')
fig.tight_layout()


## Evolution of the Copper Impurity

In [ ]:
with plt.style.context('ggplot'):
    fig, axes = plt.subplots(2, 1, sharex=True)
    data.query('IC_P_tot>0.25').plot(kind='scatter', x='pulse', y='Cu_norm_nl_IC', c='Ip', ax=axes[0], cmap="viridis", s=50, vmax=0.75, alpha=.6)
    data.query('LH_P_tot>0.25').plot(kind='scatter', x='pulse', y='Cu_norm_nl_LH', c='Ip', ax=axes[1], cmap="viridis", s=50, vmax=0.75, alpha=.6)
    [ax.set_ylim(bottom=0, top=500) for ax in axes]
    axes[0].set_ylabel('Cu / nl / IC Power [a.u.]')
    axes[1].set_ylabel('Cu / nl / LH Power [a.u.]')
    axes[-1].set_xlabel('WEST Pulse #')
    #plt.xticks(rotation=40)
    fig.suptitle('Evolution of Copper Impurity')
    fig.tight_layout()
    fig.subplots_adjust(hspace=0.2)
    axes[-1].xaxis.tick_top()

# Performances Comparaison Between Campaign

In [ ]:
# Importing Resumed Parameters
data_C4 = pd.read_csv('../WEST_C4/WEST_C4_database_resumed_parameters.csv')

In [ ]:
data_C4['IC_P_tot_MW'] = data_C4['IC_P_tot']/1e3
data_C4['IC_P_Q1_MW'] = data_C4['IC_P_Q1']/1e3
data_C4['IC_P_Q2_MW'] = data_C4['IC_P_Q2']/1e3
data_C4['IC_P_Q4_MW'] = data_C4['IC_P_Q4']/1e3
data_C4['pulse_relative'] = data_C4['pulse'] - data_C4['pulse'][0]
data_C4['P_RF'] = data['LH_P_tot'] + data['IC_P_tot']
data_C4['Ip_MA'] = data['Ip']/1e3
data_C4['Cu_norm'] = data['Cu']/data['nl']


# objet le plus avancé
data_C4['R_obj'] = data_C4[['R_Q1', 'R_Q2', 'R_Q4', 'LH_Positions']].min(axis=1)*1e3
# ROG wrt VACTH magnetic reconstruction in [mm]
data_C4['ROG_LH'] = data_C4['R_obj'] - data_C4['Rext_median']
data_C4['ROG_Q1'] = data_C4['R_Q1']*1e3 - data_C4['Rext_median']
data_C4['ROG_Q2'] = data_C4['R_Q2']*1e3 - data_C4['Rext_median']
data_C4['ROG_Q4'] = data_C4['R_Q4']*1e3 - data_C4['Rext_median']
data_C4['ROG'] = data_C4['R_obj'] - data_C4['Rext_median']
# ROG wrt NICE magnetic reconstruction in [mm]
data_C4['ROG_Q1_NICE'] = data_C4['R_Q1']*1e3 - data_C4['Rext_median_NICE']*1e3
data_C4['ROG_Q2_NICE'] = data_C4['R_Q2']*1e3 - data_C4['Rext_median_NICE']*1e3
data_C4['ROG_Q4_NICE'] = data_C4['R_Q4']*1e3 - data_C4['Rext_median_NICE']*1e3
data_C4['ROG_NICE'] = data_C4['R_obj'] - data_C4['Rext_median_NICE']*1e3


C4_after_boro_pulses = [53453, 54288, 54403, 54502, 54596, 54719, 54881, 55000, 55138, 55499, 55548, 55747, 55795]
C5_after_boro_pulses = [56244, 56596]

In [ ]:
fig, ax = plt.subplots()
data_C4_grp = data_C4.query('IC_P_tot_MW>0.1').groupby('pulse')
data_C5_grp = data.query('IC_P_tot>0.1').groupby('pulse')

data_C4_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse_relative', y='IC_P_tot_MW', 
                                      color='C0', label='C4', yerr=data_C4_grp.std().reset_index(), s=80, edgecolor='k', alpha=0.5)
data_C5_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse_relative', y='IC_P_tot', 
                                      color='C1', label='C5', yerr=data_C5_grp.std().reset_index(), s=80, edgecolor='k', alpha=0.5)
ax.set_xlabel('Relative WEST Pulse # wrt. Campaign Start')
ax.set_ylabel('Coupled Power [MW]')
for pulse in C4_after_boro_pulses:
    ax.axvline(pulse-data_C4['pulse'][0], ls='--', color='blue')
for pulse in C5_after_boro_pulses:
    ax.axvline(pulse-data['pulse'][23], ls='--', color='orange')
ax.set_xlim(left=0)
ax.set_title('Total IC Power')

In [ ]:
fig, ax = plt.subplots()
data_C4_grp = data_C4.query('IC_P_tot_MW>0.1').groupby('pulse')
data_C5_grp = data.query('IC_P_tot>0.1').groupby('pulse')

data_C4_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_P_tot_MW', 
                                      color='C0', label='C4', yerr=data_C4_grp.std().reset_index(), s=80, edgecolor='k', alpha=0.5)
data_C5_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_P_tot', 
                                      color='C1', label='C5', yerr=data_C5_grp.std().reset_index(), s=80, edgecolor='k', alpha=0.5)
ax.set_xlabel('Relative WEST Pulse #')
ax.set_ylabel('Coupled Power [MW]')
for pulse in C4_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='gray')
for pulse in C5_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='gray')
ax.set_xlim(left=data_C4['pulse'][0])
ax.set_title('Total IC Power')
fig.tight_layout()

In [ ]:
data_C4['Ip_MA']=data_C4['Ip']/1e3

In [ ]:
fig, ax = plt.subplots()
data_C4_grp = data_C4.query('IC_P_tot_MW>0.1').groupby('pulse')
data_C5_grp = data.query('IC_P_tot>0.1').groupby('pulse')

data_C4_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_P_tot_MW', cmap='tab10', colorbar=False,
                                      c='Ip_MA', label='C4', s=100, marker="s", edgecolor='k', alpha=0.8, vmin=0.5, vmax=0.8)
data_C5_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_P_tot', cmap='tab10',
                                      c='Ip', label='C5', s=100, edgecolor='k', alpha=0.8, vmin=0.5, vmax=0.8)
ax.set_xlabel('Relative WEST Pulse #')
ax.set_ylabel('Coupled Power [MW]')
for pulse in C4_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='gray')
for pulse in C5_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='gray')
ax.set_xlim(left=data_C4['pulse'][0])
ax.set_title('Total IC Power')
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
data_C4_grp = data_C4.query('P_RF>0.1').groupby('pulse')
data_C5_grp = data.query('P_RF>0.1').groupby('pulse')

data_C4_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='P_RF', 
                                      color='C0', label='C4', yerr=data_C4_grp.std().reset_index(), s=80, edgecolor='k', alpha=0.5)
data_C5_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='P_RF', 
                                      color='C1', label='C5', yerr=data_C5_grp.std().reset_index(), s=80, edgecolor='k', alpha=0.5)
ax.set_xlabel('WEST Pulse #')
ax.set_ylabel('Coupled Power [MW]')
ax.set_title('Total RF Power')
for pulse in C4_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='grey')
for pulse in C5_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='grey')
ax.set_xlim(left=data_C4['pulse'][0])

In [ ]:
fig, ax = plt.subplots()
data_C4_grp = data_C4.query('IC_P_Q2_MW>0.1').groupby('pulse')
data_C5_grp = data.query('IC_P_Q2>0.1').groupby('pulse')

data_C4_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_P_Q2_MW', 
                                      color='C0', label='C4', yerr=data_C4_grp.std().reset_index(), s=80, edgecolor='k', alpha=0.5)
data_C5_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_P_Q2', 
                                      color='C1', label='C5', yerr=data_C5_grp.std().reset_index(), s=80, edgecolor='k', alpha=0.5)
ax.set_xlabel('WEST Pulse #')
ax.set_ylabel('Coupled Power [MW]')
ax.set_title('Q2 Antenna')

for pulse in C4_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='grey')
for pulse in C5_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='grey')
ax.set_xlim(left=data_C4['pulse'][0])

In [ ]:
fig, ax = plt.subplots()
data_C4_grp = data_C4.query('IC_P_Q1_MW>0.1').groupby('pulse')
data_C5_grp = data.query('IC_P_Q1>0.1').groupby('pulse')

data_C4_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_P_Q1_MW', 
                                      color='C0', label='C4', yerr=data_C4_grp.std().reset_index(), s=60, edgecolor='k', alpha=0.5)
data_C5_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_P_Q1', 
                                      color='C1', label='C5', yerr=data_C5_grp.std().reset_index(), s=60, edgecolor='k', alpha=0.5)
ax.set_xlabel('WEST Pulse #')
ax.set_ylabel('Coupled Power [MW]')
ax.set_title('Q1 Antenna')
for pulse in C4_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='grey')
for pulse in C5_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='grey')
ax.set_xlim(left=data_C4['pulse'][0])

In [ ]:
fig, ax = plt.subplots()
data_C4_grp = data_C4.query('IC_P_Q4_MW>0.1').groupby('pulse')
data_C5_grp = data.query('IC_P_Q4>0.1').groupby('pulse')

data_C4_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_P_Q4_MW', 
                                      color='C0', label='C4', yerr=data_C4_grp.std().reset_index(), s=60, edgecolor='k', alpha=0.5)
data_C5_grp.mean().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='IC_P_Q4', 
                                      color='C1', label='C5', yerr=data_C5_grp.std().reset_index(), s=60, edgecolor='k', alpha=0.5)
ax.set_xlabel('WEST Pulse #')
ax.set_ylabel('Coupled Power [MW]')
ax.set_title('Q4 Antenna')
for pulse in C4_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='grey')
for pulse in C5_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='grey')
ax.set_xlim(left=data_C4['pulse'][0])

### Coper Evolution between C4 and C5

In [ ]:
data_C4_grp = data_C4.query('IC_P_tot_MW>0.1 & LH_P_tot < 0.1').groupby('pulse')
data_C5_grp = data.query('IC_P_tot>0.1 & LH_P_tot < 0.1').groupby('pulse')


fig, ax = plt.subplots()
data_C4_grp.max().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='Cu_norm', 
                                      color='C0', label='C4', s=60, edgecolor='k', alpha=0.5)
data_C5_grp.max().reset_index().plot(ax=ax, kind='scatter', x='pulse', y='Cu_norm', 
                                      color='C1', label='C5', s=60, edgecolor='k', alpha=0.5)
ax.set_xlabel('WEST Pulse #')
ax.set_ylabel('Max Cu/nl [a.u.]')
ax.set_title('Normalized Copper Signal during RF Power')
for pulse in C4_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='grey')
for pulse in C5_after_boro_pulses:
    ax.axvline(pulse, ls='--', color='grey')
ax.set_xlim(left=data_C4['pulse'][0])
ax.set_ylim(bottom=0)

In [ ]:
fig, ax = plt.subplots()
sns.kdeplot(ax=ax, dataset=data.query("pulse < @january_restart & IC_P_tot > 0.1"), 
                                      x=data['Isotopic Ratio INBUM04'], y=data['Cu_norm'], 
                                      shade=True, cmap='Reds', fill=True)

# Coupling Resistance vs ROG in C4

In [ ]:
data_C4['Rext_median_NICE_mm'] = data_C4['Rext_median_NICE']*1e3
data_C4['Rext_upper_NICE_mm'] = data_C4['Rext_upper_NICE']*1e3
data_C4['Rext_lower_NICE_mm'] = data_C4['Rext_lower_NICE']*1e3
# 1st object found at LFS in [mm]
data_C4['Robj'] = data_C4[['R_Q1', 'R_Q2', 'R_Q4', 'LH_Positions', 'LPA']].min(axis=1)*1e3
# ROG wrt VACTH magnetic reconstruction in [mm]
data_C4['ROG_LH'] =  - data_C4['Rext_median']
data_C4['ROG_Q1'] = data_C4['R_Q1']*1e3 - data_C4['Rext_median']
data_C4['ROG_Q2'] = data_C4['R_Q2']*1e3 - data_C4['Rext_median']
data_C4['ROG_Q4'] = data_C4['R_Q4']*1e3 - data_C4['Rext_median']
data_C4['ROG'] = data_C4['Robj'] - data_C4['Rext_median']
data_C4['ROG_upper'] = data_C4['Robj'] - data_C4['Rext_upper']
data_C4['ROG_lower'] = data_C4['Robj'] - data_C4['Rext_lower']
# ROG wrt NICE magnetic reconstruction in [mm]
data_C4['ROG_Q1_NICE'] = data_C4['R_Q1']*1e3 - data_C4['Rext_median_NICE_mm']
data_C4['ROG_Q2_NICE'] = data_C4['R_Q2']*1e3 - data_C4['Rext_median_NICE_mm']
data_C4['ROG_Q4_NICE'] = data_C4['R_Q4']*1e3 - data_C4['Rext_median_NICE_mm']
data_C4['ROG_NICE'] = data_C4['Robj'] - data_C4['Rext_median_NICE_mm']
data_C4['ROG_upper_NICE'] = data_C4['Robj'] - data_C4['Rext_upper_NICE_mm']
data_C4['ROG_lower_NICE'] = data_C4['Robj'] - data_C4['Rext_lower_NICE_mm']

In [ ]:
# Define USN.LSN plasma configuration
data_C4['Xpoint'] = 'LSN'  # default plasma are LSN, only a few listed below are USN
USN_list = np.r_[55227, 55174, 55173, 55172, 55171, 55170]
USN_list = np.append(USN_list, np.arange(54882, 54892))
USN_list = np.append(USN_list, np.arange(54894, 54903))
USN_list = np.append(USN_list, np.load('WEST_C4_USN_pulse.npy'))  # List by J.Morales
USN_list = np.unique(USN_list)
data_C4.loc[data_C4['pulse'].isin(USN_list), 'Xpoint'] = 'USN'

In [ ]:
from scipy.stats import kde

In [ ]:
####### C4
xy = np.r_[
    data_C4.query("IC_P_Q1 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='LSN'")[['ROG_NICE','IC_Rc_Q1_avg']].dropna().values,
    data_C4.query("IC_P_Q2 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='LSN'")[['ROG_NICE','IC_Rc_Q2_avg']].dropna().values,
    data_C4.query("IC_P_Q4 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='LSN'")[['ROG_NICE','IC_Rc_Q4_avg']].dropna().values,
]
x=xy[:,0]; y=xy[:,1]

# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
nbins=200

k = kde.gaussian_kde([x,y])
# complex bins number means end is inclusive
xi_C4_NICE, yi_C4_NICE = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi_C4_NICE = k(np.vstack([xi_C4_NICE.flatten(), yi_C4_NICE.flatten()]))

xy = np.r_[
    data_C4.query("IC_P_Q1 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='LSN'")[['ROG','IC_Rc_Q1_avg']].dropna().values,
    data_C4.query("IC_P_Q2 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='LSN'")[['ROG','IC_Rc_Q2_avg']].dropna().values,
    data_C4.query("IC_P_Q4 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='LSN'")[['ROG','IC_Rc_Q4_avg']].dropna().values,
]
x=xy[:,0]; y=xy[:,1]

# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
nbins=200

k = kde.gaussian_kde([x,y])
# complex bins number means end is inclusive
xi_C4_VACTH, yi_C4_VACTH = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi_C4_VACTH = k(np.vstack([xi_C4_VACTH.flatten(), yi_C4_VACTH.flatten()]))


### C5
xy = np.r_[
    data.query("IC_P_Q1 > 0.1 & pulse>=@phase_control_OK & ROG_NICE > 0 & Xpoint=='LSN'")[['ROG_NICE','IC_Rc_Q1_avg']].dropna().values,
    data.query("IC_P_Q2 > 0.1 & pulse>=@phase_control_OK & ROG_NICE > 0 & Xpoint=='LSN'")[['ROG_NICE','IC_Rc_Q2_avg']].dropna().values,
    data.query("IC_P_Q4 > 0.1 & pulse>=@phase_control_OK & ROG_NICE > 0 & Xpoint=='LSN'")[['ROG_NICE','IC_Rc_Q4_avg']].dropna().values,
]
x=xy[:,0]; y=xy[:,1]

# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
nbins=400

k = kde.gaussian_kde([x,y])
# complex bins number means end is inclusive
xi_C5_NICE, yi_C5_NICE = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi_C5_NICE = k(np.vstack([xi_C5_NICE.flatten(), yi_C5_NICE.flatten()]))


xy = np.r_[
    data.query("IC_P_Q1 > 0.1 & pulse>=@phase_control_OK & ROG > 0 & Xpoint=='LSN'")[['ROG','IC_Rc_Q1_avg']].dropna().values,
    data.query("IC_P_Q2 > 0.1 & pulse>=@phase_control_OK & ROG > 0 & Xpoint=='LSN'")[['ROG','IC_Rc_Q2_avg']].dropna().values,
    data.query("IC_P_Q4 > 0.1 & pulse>=@phase_control_OK & ROG > 0 & Xpoint=='LSN'")[['ROG','IC_Rc_Q4_avg']].dropna().values,
]
x=xy[:,0]; y=xy[:,1]

# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
nbins=400

k = kde.gaussian_kde([x,y])
# complex bins number means end is inclusive
xi_C5_VACTH, yi_C5_VACTH = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi_C5_VACTH = k(np.vstack([xi_C5_VACTH.flatten(), yi_C5_VACTH.flatten()]))

In [ ]:
fig, ax = plt.subplots()
ax.contour(xi_C4_NICE, yi_C4_NICE, zi_C4_NICE.reshape(xi_C4_NICE.shape), cmap='Blues', label='C4 - NICE', lw=2)
ax.contour(xi_C4_VACTH, yi_C4_VACTH, zi_C4_VACTH.reshape(xi_C4_VACTH.shape), cmap='Purples', label='C4 - VACTH', lw=2)

ax.contour(xi_C5_NICE, yi_C5_NICE, zi_C5_NICE.reshape(xi_C5_NICE.shape), cmap='Oranges', label='C5 - NICE', lw=2)
ax.contour(xi_C5_VACTH, yi_C5_VACTH, zi_C5_VACTH.reshape(xi_C5_VACTH.shape), cmap='Reds', label='C5 - VACTH', lw=2)

#ax.scatter(x,y,color='gray', alpha=0.4)
ax.set_ylim(bottom=0, top=1.75)
ax.set_xlim(left=0, right=50) 
ax.legend(('test'))
ax.set_title('LSN Plasmas')


In [ ]:
fig, axes = plt.subplots(2,1,sharex=True)
axes[0].contour(xi_C4_VACTH, yi_C4_VACTH, zi_C4_VACTH.reshape(xi_C4_VACTH.shape), cmap='Blues', label='C4 - VACTH', lw=2)
axes[0].contour(xi_C5_VACTH, yi_C5_VACTH, zi_C5_VACTH.reshape(xi_C5_VACTH.shape), cmap='Reds', label='C5 - VACTH', lw=2)

axes[1].contour(xi_C4_NICE, yi_C4_NICE, zi_C4_NICE.reshape(xi_C4_NICE.shape), cmap='Blues', label='C4 - NICE', lw=2)
axes[1].contour(xi_C5_NICE, yi_C5_NICE, zi_C5_NICE.reshape(xi_C5_NICE.shape), cmap='Reds', label='C5 - NICE', lw=2)


for ax in axes:
    ax.set_ylim(bottom=0, top=1.75)
    ax.set_xlim(left=0, right=50) 
    ax.legend(('test'))
    ax.set_ylabel('Average $R_c$ [Ohm]')


fig.suptitle('LSN Plasmas')
ax.set_xlabel('ROG [mm]')
fig.tight_layout()
fig.subplots_adjust(hspace=0.1)

In [ ]:
fig, axes = plt.subplots(2, 2, sharex=True, sharey=True)
axes[0,0].contourf(xi_C4_NICE, yi_C4_NICE, zi_C4_NICE.reshape(xi_C4_NICE.shape), cmap='Purples')
axes[0,1].contourf(xi_C4_VACTH, yi_C4_VACTH, zi_C4_VACTH.reshape(xi_C4_VACTH.shape), cmap='Purples')

axes[1,0].contourf(xi_C5_NICE, yi_C5_NICE, zi_C5_NICE.reshape(xi_C5_NICE.shape), cmap='Oranges')
axes[1,1].contourf(xi_C5_VACTH, yi_C5_VACTH, zi_C5_VACTH.reshape(xi_C5_VACTH.shape), cmap='Oranges')

#ax.scatter(x,y,color='gray', alpha=0.4)
[ax.set_ylim(bottom=0, top=2.5) for ax in axes.flatten()]
[ax.set_xlim(left=0, right=50) for ax in axes.flatten()]
fig.suptitle('LSN Plasmas')

In [ ]:
####### C4
xy = np.r_[
    data_C4.query("IC_P_Q1 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='USN'")[['ROG_NICE','IC_Rc_Q1_avg']].dropna().values,
    data_C4.query("IC_P_Q2 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='USN'")[['ROG_NICE','IC_Rc_Q2_avg']].dropna().values,
    data_C4.query("IC_P_Q4 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='USN'")[['ROG_NICE','IC_Rc_Q4_avg']].dropna().values,
]
x=xy[:,0]; y=xy[:,1]

# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
nbins=400

k = kde.gaussian_kde([x,y])
# complex bins number means end is inclusive
xi_C4_NICE, yi_C4_NICE = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi_C4_NICE = k(np.vstack([xi_C4_NICE.flatten(), yi_C4_NICE.flatten()]))

xy = np.r_[
    data_C4.query("IC_P_Q1 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN'")[['ROG','IC_Rc_Q1_avg']].dropna().values,
    data_C4.query("IC_P_Q2 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN'")[['ROG','IC_Rc_Q2_avg']].dropna().values,
    data_C4.query("IC_P_Q4 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN'")[['ROG','IC_Rc_Q4_avg']].dropna().values,
]
x=xy[:,0]; y=xy[:,1]

# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
nbins=400

k = kde.gaussian_kde([x,y])
# complex bins number means end is inclusive
xi_C4_VACTH, yi_C4_VACTH = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi_C4_VACTH = k(np.vstack([xi_C4_VACTH.flatten(), yi_C4_VACTH.flatten()]))


### C5
xy = np.r_[
    data.query("IC_P_Q1 > 0.1 & pulse>=@phase_control_OK & ROG_NICE > 0 & Xpoint=='USN'")[['ROG_NICE','IC_Rc_Q1_avg']].dropna().values,
    data.query("IC_P_Q2 > 0.1 & pulse>=@phase_control_OK & ROG_NICE > 0 & Xpoint=='USN'")[['ROG_NICE','IC_Rc_Q2_avg']].dropna().values,
    data.query("IC_P_Q4 > 0.1 & pulse>=@phase_control_OK & ROG_NICE > 0 & Xpoint=='USN'")[['ROG_NICE','IC_Rc_Q4_avg']].dropna().values,
]
x=xy[:,0]; y=xy[:,1]

# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
nbins=400

k = kde.gaussian_kde([x,y])
# complex bins number means end is inclusive
xi_C5_NICE, yi_C5_NICE = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi_C5_NICE = k(np.vstack([xi_C5_NICE.flatten(), yi_C5_NICE.flatten()]))


xy = np.r_[
    data.query("IC_P_Q1 > 0.1 & pulse>=@phase_control_OK & ROG > 0 & Xpoint=='USN'")[['ROG','IC_Rc_Q1_avg']].dropna().values,
    data.query("IC_P_Q2 > 0.1 & pulse>=@phase_control_OK & ROG > 0 & Xpoint=='USN'")[['ROG','IC_Rc_Q2_avg']].dropna().values,
    data.query("IC_P_Q4 > 0.1 & pulse>=@phase_control_OK & ROG > 0 & Xpoint=='USN'")[['ROG','IC_Rc_Q4_avg']].dropna().values,
]
x=xy[:,0]; y=xy[:,1]

# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
nbins=400

k = kde.gaussian_kde([x,y])
# complex bins number means end is inclusive
xi_C5_VACTH, yi_C5_VACTH = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi_C5_VACTH = k(np.vstack([xi_C5_VACTH.flatten(), yi_C5_VACTH.flatten()]))

In [ ]:
fig, axes = plt.subplots(2,1,sharex=True)
axes[0].contour(xi_C4_VACTH, yi_C4_VACTH, zi_C4_VACTH.reshape(xi_C4_VACTH.shape), cmap='Blues', label='C4 - VACTH', lw=2)
axes[0].contour(xi_C5_VACTH, yi_C5_VACTH, zi_C5_VACTH.reshape(xi_C5_VACTH.shape), cmap='Reds', label='C5 - VACTH', lw=2)

axes[1].contour(xi_C4_NICE, yi_C4_NICE, zi_C4_NICE.reshape(xi_C4_NICE.shape), cmap='Blues', label='C4 - NICE', lw=2)
axes[1].contour(xi_C5_NICE, yi_C5_NICE, zi_C5_NICE.reshape(xi_C5_NICE.shape), cmap='Reds', label='C5 - NICE', lw=2)


for ax in axes:
    ax.set_ylim(bottom=0, top=1.75)
    ax.set_xlim(left=0, right=50) 
    ax.legend(('test'))
    ax.set_ylabel('Average $R_c$ [Ohm]')


fig.suptitle('USN Plasmas')
ax.set_xlabel('ROG [mm]')
fig.tight_layout()
fig.subplots_adjust(hspace=0.1)

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True)
alp=0.5
data_C4.query("IC_P_Q1 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='LSN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q1_avg', label='C4', alpha=alp)
data_C4.query("IC_P_Q2 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='LSN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q2_avg', alpha=alp)
data_C4.query("IC_P_Q4 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='LSN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q4_avg', alpha=alp)
data.query("IC_P_Q1 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='LSN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q1_avg', color='C1', label='C5', alpha=alp)
data.query("IC_P_Q2 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='LSN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q2_avg', color='C1', alpha=alp)
data.query("IC_P_Q4 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='LSN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q4_avg', color='C1', alpha=alp) 

data_C4.query("IC_P_Q1 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='LSN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q1_avg', label='C4', alpha=alp)
data_C4.query("IC_P_Q2 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='LSN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q2_avg', alpha=alp)
data_C4.query("IC_P_Q4 > 0.3 & ROG_NICE > 0 & ROG < 100 & Xpoint=='LSN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q4_avg', alpha=alp)
data.query("IC_P_Q1 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='LSN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q1_avg', color='C1', label='C5', alpha=alp)
data.query("IC_P_Q2 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='LSN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q2_avg', color='C1', alpha=alp)
data.query("IC_P_Q4 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='LSN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q4_avg', color='C1', alpha=alp)

for a in ax:
    a.set_ylim(bottom=0, top=2.5)
    a.set_xlim(left=0, right=70) 
    a.set_ylabel('Average $R_c$ [Ohm]')

fig.suptitle('LSN Plasmas')
ax[-1].set_xlabel('ROG [mm]')
fig.tight_layout()
fig.subplots_adjust(hspace=0.1)

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True)

data_C4.query("IC_P_Q1 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q1_avg', label='C4')
data_C4.query("IC_P_Q2 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q2_avg')
data_C4.query("IC_P_Q4 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q4_avg')
data.query("IC_P_Q1 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q1_avg', color='C1', label='C5')
data.query("IC_P_Q2 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q2_avg', color='C1')
data.query("IC_P_Q4 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN'").plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q4_avg', color='C1') 

data_C4.query("IC_P_Q1 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='USN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q1_avg', label='C4')
data_C4.query("IC_P_Q2 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='USN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q2_avg')
data_C4.query("IC_P_Q4 > 0.3 & ROG_NICE > 0 & ROG < 100 & Xpoint=='USN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q4_avg')
data.query("IC_P_Q1 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='USN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q1_avg', color='C1', label='C5')
data.query("IC_P_Q2 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='USN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q2_avg', color='C1')
data.query("IC_P_Q4 > 0.3 & ROG_NICE > 0 & ROG_NICE < 100 & Xpoint=='USN'").plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q4_avg', color='C1')

for a in ax:
    a.set_ylim(bottom=0, top=2.5)
    a.set_xlim(left=0, right=70) 
    a.set_ylabel('Average $R_c$ [Ohm]')

fig.suptitle('USN Plasmas')
ax[-1].set_xlabel('ROG [mm]')
fig.tight_layout()
fig.subplots_adjust(hspace=0.1)

In [ ]:

fig, ax = plt.subplots(2,1,sharex=True)
alp=0.5
Pth = 0.3
_C4_Q1 = data_C4.query("IC_P_Q1 > @Pth & ROG > 0 & ROG < 100 & Xpoint=='LSN' & MHD<60").groupby('pulse')
_C4_Q2 = data_C4.query("IC_P_Q2 > @Pth & ROG > 0 & ROG < 100 & Xpoint=='LSN' & MHD<60").groupby('pulse')
_C4_Q4 = data_C4.query("IC_P_Q4 > @Pth & ROG > 0 & ROG < 100 & Xpoint=='LSN' & MHD<60").groupby('pulse')
_C5_Q1 = data.query("IC_P_Q1 > @Pth & ROG > 0 & ROG < 100 & Xpoint=='LSN' & MHD<60").groupby('pulse')
_C5_Q2 = data.query("IC_P_Q2 > @Pth & ROG > 0 & ROG < 100 & Xpoint=='LSN' & MHD<60").groupby('pulse')
_C5_Q4 = data.query("IC_P_Q4 > @Pth & ROG > 0 & ROG < 100 & Xpoint=='LSN' & MHD<60").groupby('pulse')

_C4_Q1.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q1_avg', label='C4', color='C0', alpha=alp, yerr=_C4_Q1.std(), edgecolor='k', s=40, xerr=_C4_Q1.std())
_C4_Q2.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q2_avg', color='C0', alpha=alp, yerr=_C4_Q2.std(), edgecolor='k', s=40, xerr=_C4_Q2.std())
_C4_Q4.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q4_avg', color='C0', alpha=alp, yerr=_C4_Q4.std(), edgecolor='k', s=40, xerr=_C4_Q4.std())
_C5_Q1.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q1_avg', label='C5', color='C1', alpha=alp, yerr=_C5_Q1.std(), edgecolor='k', s=40, xerr=_C5_Q1.std())
_C5_Q2.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q2_avg', color='C1', alpha=alp, yerr=_C5_Q2.std(), edgecolor='k', s=40, xerr=_C5_Q2.std())
_C5_Q4.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q4_avg', color='C1', alpha=alp, yerr=_C5_Q4.std(), edgecolor='k', s=40, xerr=_C5_Q4.std())

_C4_Q1.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q1_avg', label='C4', color='C0', alpha=alp, yerr=_C4_Q1.std(), edgecolor='k', s=40, xerr=_C4_Q1.std())
_C4_Q2.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q2_avg', color='C0', alpha=alp, yerr=_C4_Q2.std(), edgecolor='k', s=40, xerr=_C4_Q2.std())
_C4_Q4.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q4_avg', color='C0', alpha=alp, yerr=_C4_Q4.std(), edgecolor='k', s=40, xerr=_C4_Q4.std())
_C5_Q1.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q1_avg', label='C5', color='C1', alpha=alp, yerr=_C5_Q1.std(), edgecolor='k', s=40, xerr=_C5_Q1.std())
_C5_Q2.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q2_avg', color='C1', alpha=alp, yerr=_C5_Q2.std(), edgecolor='k', s=40, xerr=_C5_Q2.std())
_C5_Q4.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q4_avg', color='C1', alpha=alp, yerr=_C5_Q4.std(), edgecolor='k', s=40, xerr=_C5_Q4.std())

for a in ax:
    a.set_ylim(bottom=0, top=2.5)
    a.set_xlim(left=0, right=70) 
    a.set_ylabel('$\overline{R_c}$ [Ohm]')

fig.suptitle('LSN Plasmas ; $P_{IC}>'+str(Pth)+'$MW')
ax[-1].set_xlabel('ROG [mm]')
fig.tight_layout()
fig.subplots_adjust(hspace=0.1)

In [ ]:

fig, ax = plt.subplots(2,1,sharex=True)
alp=0.5
_C4_Q1 = data_C4.query("IC_P_Q1 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN' & MHD<60").groupby('pulse')
_C4_Q2 = data_C4.query("IC_P_Q2 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN' & MHD<60").groupby('pulse')
_C4_Q4 = data_C4.query("IC_P_Q4 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN' & MHD<60").groupby('pulse')
_C5_Q1 = data.query("IC_P_Q1 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN' & MHD<60").groupby('pulse')
_C5_Q2 = data.query("IC_P_Q2 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN' & MHD<60").groupby('pulse')
_C5_Q4 = data.query("IC_P_Q4 > 0.3 & ROG > 0 & ROG < 100 & Xpoint=='USN' & MHD<60").groupby('pulse')

_C4_Q1.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q1_avg', label='C4', color='C0', alpha=alp, yerr=_C4_Q1.std(), edgecolor='k', s=40, xerr=_C4_Q1.std())
_C4_Q2.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q2_avg', color='C0', alpha=alp, yerr=_C4_Q2.std(), edgecolor='k', s=40, xerr=_C4_Q2.std())
_C4_Q4.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q4_avg', color='C0', alpha=alp, yerr=_C4_Q4.std(), edgecolor='k', s=40, xerr=_C4_Q4.std())
_C5_Q1.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q1_avg', label='C5', color='C1', alpha=alp, yerr=_C5_Q1.std(), edgecolor='k', s=40, xerr=_C5_Q1.std())
_C5_Q2.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q2_avg', color='C1', alpha=alp, yerr=_C5_Q2.std(), edgecolor='k', s=40, xerr=_C5_Q2.std())
_C5_Q4.mean().plot(ax=ax[0], kind='scatter', x='ROG', y='IC_Rc_Q4_avg', color='C1', alpha=alp, yerr=_C5_Q4.std(), edgecolor='k', s=40, xerr=_C5_Q4.std())

_C4_Q1.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q1_avg', label='C4', color='C0', alpha=alp, yerr=_C4_Q1.std(), edgecolor='k', s=40, xerr=_C4_Q1.std())
_C4_Q2.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q2_avg', color='C0', alpha=alp, yerr=_C4_Q2.std(), edgecolor='k', s=40, xerr=_C4_Q2.std())
_C4_Q4.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q4_avg', color='C0', alpha=alp, yerr=_C4_Q4.std(), edgecolor='k', s=40, xerr=_C4_Q4.std())
_C5_Q1.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q1_avg', label='C5', color='C1', alpha=alp, yerr=_C5_Q1.std(), edgecolor='k', s=40, xerr=_C5_Q1.std())
_C5_Q2.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q2_avg', color='C1', alpha=alp, yerr=_C5_Q2.std(), edgecolor='k', s=40, xerr=_C5_Q2.std())
_C5_Q4.mean().plot(ax=ax[1], kind='scatter', x='ROG_NICE', y='IC_Rc_Q4_avg', color='C1', alpha=alp, yerr=_C5_Q4.std(), edgecolor='k', s=40, xerr=_C5_Q4.std())

for a in ax:
    a.set_ylim(bottom=0, top=2.5)
    a.set_xlim(left=0, right=70) 
    a.set_ylabel('$\overline{R_c}$ [Ohm]')

fig.suptitle('USN Plasmas ; $P_{IC}>0.3$MW')
ax[-1].set_xlabel('ROG [mm]')
fig.tight_layout()
fig.subplots_adjust(hspace=0.1)